# Answering Business Questions using SQL

## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type in ("type", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |


The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [9]:
%%sql
WITH usa_tracks_sold AS
   (
    SELECT il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name,
    COUNT(uts.invoice_line_id) AS tracks_sold,
    CAST(COUNT(uts.invoice_line_id) AS FLOAT) / (SELECT COUNT(*) 
        FROM usa_tracks_sold) AS percentage_sold
FROM usa_tracks_sold AS uts
INNER JOIN track AS t ON t.track_id = uts.track_id
INNER JOIN genre AS g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Done.


name,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [13]:
%%sql
WITH customer_support_rep_sales AS
    (
    SELECT
        i.customer_id,
        c.support_rep_id,
        SUM(i.total) AS total
    FROM invoice AS i
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    GROUP BY 1, 2
    )
    
SELECT
    e.first_name || " " || e.last_name AS employee,
    e.title,
    SUM(csrs.total) AS total_sales
FROM customer_support_rep_sales AS csrs
INNER JOIN employee AS e ON e.employee_id = csrs.support_rep_id
GROUP BY 1;

Done.


employee,title,total_sales
Jane Peacock,Sales Support Agent,1731.5099999999998
Margaret Park,Sales Support Agent,1584.0000000000002
Steve Johnson,Sales Support Agent,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country


Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we should group these customers as "Other" in our analysis. We can use the following 'trick' to force the ordering of "Other" to last in our analysis.

In [14]:
%%sql
WITH country_or_other AS
    (
    SELECT 
        CASE
            WHEN (
            SELECT COUNT(*)
            FROM customer
            WHERE country = c.country
            ) = 1
        THEN "Other"
        ELSE c.country
    END AS country,
    c.customer_id,
    il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        COUNT(distinct customer_id) AS customers,
        SUM(unit_price) AS total_sales,
        SUM(unit_price) / COUNT(distinct customer_id) AS customer_lifetime_value,
        SUM(unit_price) / COUNT(distinct invoice_id) AS average_order,
        CASE
            WHEN country = "Other" 
            THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [15]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id AS invoice_id,
         MIN(il.track_id) AS first_track_id
     FROM invoice_line AS il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) AS number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
        SELECT COUNT(*) FROM invoice
        ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track AS t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line AS il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line AS il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track AS ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, we would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.